In [1]:
!pip install setfit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.2/75.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 11.1 MB/s eta 0:00:00


In [2]:
# SETFIT ERROR as of December 2024
# AttributeError: 'CallbackHandler' object has no attribute 'tokenizer'
# https://github.com/huggingface/setfit/issues/564
# solution is to install older version of transformers
!pip install transformers==4.45.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 70.7 MB/s eta 0:00:00:00:010:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.3
    Uninstalling transformers-4.46.3:
      Successfully uninstalled transformers-4.46.3


In [3]:
from setfit import SetFitModel

model = SetFitModel.from_pretrained("BAAI/bge-small-en-v1.5")

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [4]:
from datasets import load_dataset

dataset = load_dataset("data-is-better-together/10k_prompts_ranked")

dataset

README.md:   0%|          | 0.00/11.4k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.58M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10331 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['prompt', 'quality', 'metadata', 'avg_rating', 'num_responses', 'agreement_ratio', 'raw_responses', 'kind', 'cluster_description', 'topic'],
        num_rows: 10331
    })
})

In [5]:
dataset_tt = dataset["train"].train_test_split(test_size=0.5)

In [6]:
dataset_tt

DatasetDict({
    train: Dataset({
        features: ['prompt', 'quality', 'metadata', 'avg_rating', 'num_responses', 'agreement_ratio', 'raw_responses', 'kind', 'cluster_description', 'topic'],
        num_rows: 5165
    })
    test: Dataset({
        features: ['prompt', 'quality', 'metadata', 'avg_rating', 'num_responses', 'agreement_ratio', 'raw_responses', 'kind', 'cluster_description', 'topic'],
        num_rows: 5166
    })
})

In [7]:
from setfit import sample_dataset

# Mimic only having a few labeled examples from each class

- here i'm taking 20 of each class (`topic` is the `label` in our current dataset, will rename later)
- so `sample_dataset` from setfit should take 20 **of each possible label** so we should see 20 * 10 = 200 samples in train dataset below (since there are 10 labels in this dataset)

In [8]:
train_dataset = sample_dataset(dataset_tt["train"], label_column="topic", num_samples=20)

In [9]:
train_dataset

Dataset({
    features: ['prompt', 'quality', 'metadata', 'avg_rating', 'num_responses', 'agreement_ratio', 'raw_responses', 'kind', 'cluster_description', 'topic'],
    num_rows: 200
})

In [10]:
test_dataset = dataset_tt["test"]

In [11]:
test_dataset

Dataset({
    features: ['prompt', 'quality', 'metadata', 'avg_rating', 'num_responses', 'agreement_ratio', 'raw_responses', 'kind', 'cluster_description', 'topic'],
    num_rows: 5166
})

In [15]:
from setfit import TrainingArguments

args = TrainingArguments(
    batch_size=32,
    #num_epochs=1, # adjusted down from 10
    max_steps=50,
    report_to="none",
)

"""
The num_epochs and max_steps arguments are frequently used to increase and decrease the number of total training steps. Consider that with SetFit, better performance is reached with more data, not more training! Don’t be afraid to train for less than 1 epoch if you have a lot of data.
"""

'\nThe num_epochs and max_steps arguments are frequently used to increase and decrease the number of total training steps. Consider that with SetFit, better performance is reached with more data, not more training! Don’t be afraid to train for less than 1 epoch if you have a lot of data.\n'

In [16]:
from setfit import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    column_mapping={"prompt":"text", "topic":"label"} # <--- SETFIT EXPECTS text AND label RESPECTIVELY
)

Applying column mapping to the training dataset


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [17]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [18]:
import wandb
wandb.init(mode="disabled")

/opt/conda/lib/python3.10/site-packages/notebook/utils.py:280: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return LooseVersion(v) >= LooseVersion(check)


In [19]:
trainer.train()

***** Running training *****
  Num unique pairs = 1600
  Batch size = 32
  Num epochs = 1
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):


Step,Training Loss
1,0.215200
50,0.218200


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

In [20]:
trainer.evaluate(test_dataset)

Applying column mapping to the evaluation dataset
***** Running evaluation *****


Batches:   0%|          | 0/162 [00:00<?, ?it/s]

{'accuracy': 0.4742547425474255}

In [21]:
model.predict(["write a new novel for me", "what is the best place to go to visit in italy?"])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

array(['Literature and Arts', 'Travel and Leisure'], dtype='<U22')